In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import pickle
from time import localtime, strftime

In [ ]:
pngdir = 'preprocessed_png'
wildcard = '*.png'
mic_list = sorted(list(Path(pngdir).glob(wildcard)))
label_dict = {}
label_dict_name_prefix = ''

In [ ]:
def read_png(file):
    with open(file, "rb") as image:
        f = image.read()
        b = bytearray(f)
    return b

In [ ]:
def update_label():
    global rad_buttons
    ori_mic_name = mic_list[mic_index].name.replace('.png','.mrc')
    feature_list = [but.value for but in rad_buttons]
    label_dict[ori_mic_name] = feature_list

In [ ]:
def retrieve_label():
    global rad_buttons
    ori_mic_name = mic_list[mic_index].name.replace('.png','.mrc')
    if ori_mic_name in label_dict.keys():
        feature_list = label_dict[ori_mic_name]
    else:
        feature_list = ['no_film','no_crack_drift','not_crystalline','minor_contamination']
    for but,label in zip(rad_buttons,feature_list): but.value = label

In [ ]:
def prev_mic(b):
    global mic_index, mic_widget, progress
    update_label()
    mic_index -= 1
    if mic_index < 0:
        mic_index = 0
        return
    retrieve_label()
    mic_widget.value = read_png(mic_list[mic_index])
    progress.value = f'Current Micrograph: {mic_index+1} / {len(mic_list)}'

In [ ]:
def next_mic(b):
    global mic_index, mic_widget, progress
    update_label()
    mic_index += 1
    if mic_index >= len(mic_list):
        mic_index = len(mic_list)-1
        return
    retrieve_label()
    mic_widget.value = read_png(mic_list[mic_index])
    progress.value = f'Current Micrograph: {mic_index+1} / {len(mic_list)}'

In [ ]:
def save_label(b):
    global label_dict_name_prefix
    if label_dict_name_prefix != '' and label_dict_name_prefix[-1] != '_': label_dict_name_prefix += '_'
    with open(f'{label_dict_name_prefix}label_dict_{len(label_dict)}_{strftime("%m_%d_%H_%M_%S", localtime())}.pkl', 'wb') as handle:
        pickle.dump(label_dict, handle)

In [ ]:
mic_index = 0

progress = widgets.HTML(value=f'Current Micrograph: {mic_index+1} / {len(mic_list)}')
display(progress)

mic_widget = widgets.Image(value=read_png(mic_list[mic_index]),format='png')
display(mic_widget)

prev_button = widgets.Button(description='Previous')
prev_button.on_click(prev_mic)

next_button = widgets.Button(description='Next')
next_button.on_click(next_mic)

display(widgets.HBox([prev_button, next_button]))

rad_buttons = [widgets.RadioButtons(options=['no_film','minor_film','major_film','film'],disabled=False),
               widgets.RadioButtons(options=['no_crack_drift','crack_drift_empty'],disabled=False),
               widgets.RadioButtons(options=['not_crystalline','minor_crystalline','major_crystalline'],disabled=False),
               widgets.RadioButtons(options=['minor_contamination','major_contamination'],disabled=False)]
for but in rad_buttons:display(but)
retrieve_label()

save_button = widgets.Button(description='Save')
save_button.on_click(save_label)

display(save_button)